## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
import torchvision.models as models
from torchvision.models import resnet50
from torchvision.models import inception_v3
from warmup_scheduler import GradualWarmupScheduler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':5e-4,
    'BATCH_SIZE':64,
    'SEED':909,
    'NUM_WORKERS':8,
    'WEIGHT_DECAY':1e-5,
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Custom Dataset

In [5]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [6]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(always_apply=False, p=0.7),
                            A.RandomResizedCrop(always_apply=False, p=0.5, height=CFG["IMG_SIZE"], width=CFG["IMG_SIZE"], scale=(0.7, 1.0), ratio=(0.5, 2.0), interpolation=3),
                            # A.CLAHE(always_apply=False, p=0.5, clip_limit=(1, 4), tile_grid_size=(8, 8)),
                            # A.AdvancedBlur(always_apply=False, p=0.5, blur_limit=(3, 7), sigmaX_limit=(0.2, 1.0), sigmaY_limit=(0.2, 1.0), rotate_limit=(-90, 90), beta_limit=(0.5, 8.0), noise_limit=(0.9, 1.1)),
                            A.PixelDropout(always_apply=False, p=0.5, dropout_prob=0.01, per_channel=0, drop_value=(0, 0, 0), mask_drop_value=None),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

## Model Define

In [7]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(ResNet, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('resnetv2_101', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))
       
    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [8]:
class ResNetV2(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(ResNetV2, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('resnetv2_50x3_bitm', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))
       
    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [9]:
class EfficientNetB3(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(EfficientNetB3, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('efficientnet_b3', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [10]:
class EfficientNetB7(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(EfficientNetB7, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('efficientnet_b7', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [11]:
class EfficientNetB8(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(EfficientNetB8, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('efficientnet_b8', pretrained=self.pretrained, num_classes=self.num_classes)
        # self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
        #                        nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        # x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [12]:
class Vit224(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(Vit224, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('vit_base_patch16_224', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [13]:
class Maxvit224(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(Maxvit224, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('maxvit_rmlp_small_rw_224', pretrained=self.pretrained, num_classes=self.num_classes)
        # self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
        #                       nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        # x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [14]:
class Densenet(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(Densenet, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('densenet201', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [15]:
class Convnext(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super(Convnext, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        self.model = timm.create_model('convnext_base', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, num_classes))

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

## Train

In [16]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    # Warm Up
    warmup_epochs = int(CFG["EPOCHS"] * 0.15)
    warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=warmup_epochs, after_scheduler=scheduler)
    
    best_val_loss = 999999
    best_val_f1 = 0
    best_model = None
    
    # Early Stop
    patience_limit = 3
    patience = 0
    
    for epoch in range(1, CFG['EPOCHS']+1):       
        if epoch <= warmup_epochs:
            warmup.step()
        
        torch.cuda.empty_cache()
        model.train()
        
        train_loss = []        
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc, _val_f1 = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}] Val F1 : [{_val_f1:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_f1 < _val_f1:
            best_val_loss = _val_loss
            best_val_f1 = _val_f1
            best_model = model
            patience = 0
        else:
            patience += 1
            if patience >= patience_limit:
                break

    print(f'Best Loss : [{best_val_loss:.5f}] Best ACC : [{best_val_f1:.5f}]')
    return best_model

In [17]:
def validation(model, criterion, val_loader, device):
    model.eval()
    
    val_loss = []
    val_acc = []
    val_f1 = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            batch_f1 = f1_score(labels, preds, average='macro')
            
            val_f1.append(batch_f1)
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
        _val_f1 = np.mean(val_f1)
    
    return _val_loss, _val_acc, _val_f1

## Data Load

In [18]:
df = pd.read_csv('./data/train_back.csv')
df['img_path'] = df['img_path'].apply(lambda x: x.replace('./train/', './data/train/'))

## Train / Valid Split

In [19]:
df = df.sample(frac=1)
train_len = int(len(df) * 0.8)

In [20]:
train_x = df[:train_len]
valid_x = df[train_len:]

## Data Preprocessing

In [21]:
def get_labels(df):
    return df.iloc[:,2:].values

In [22]:
train_labels = get_labels(train_x)
valid_labels = get_labels(valid_x)

## Run

In [23]:
train_dataset = CustomDataset(train_x['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

val_dataset = CustomDataset(valid_x['img_path'].values, valid_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

model = Densenet()
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"], weight_decay=CFG["WEIGHT_DECAY"])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0, )
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[ int(CFG["EPOCHS"]*0.2), int(CFG["EPOCHS"]*0.4), int(CFG["EPOCHS"]*0.6)], gamma=0.7)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.33151] Val Loss : [0.15819] Val ACC : [0.93834] Val F1 : [0.92840]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [23], Train Loss : [0.02790] Val Loss : [0.04505] Val ACC : [0.98462] Val F1 : [0.98097]


  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [24], Train Loss : [0.02930] Val Loss : [0.04013] Val ACC : [0.98535] Val F1 : [0.98197]
Best Loss : [0.03694] Best ACC : [0.98367]


## Inference

In [24]:
test = pd.read_csv('./data/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: x.replace('./test/', './data/test/'))

In [25]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [26]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

## TTA (Test Time Augmentation)

In [32]:
import ttach as tta

In [33]:
tta_transforms = tta.Compose(
    [   
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 5, 355]),
        tta.Multiply(factors=[0.9, 1, 1.1])
    ]
)

In [34]:
tta_model = tta.ClassificationTTAWrapper(infer_model, tta_transforms)

In [35]:
# preds = inference(infer_model, test_loader, device)
preds_tta = inference(tta_model, test_loader, device)

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

## Submission

In [ ]:
file_name = 'Densenet201_9836_img224'

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')

In [37]:
submit.iloc[:,1:] = preds
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,1,0,0,0,1,0,0,1,1
2,TEST_00002,1,1,0,0,1,1,0,1,0,1
3,TEST_00003,1,1,0,0,0,1,0,1,1,0
4,TEST_00004,0,0,0,0,1,0,0,0,0,0


In [38]:
submit.to_csv(f'./submits/{file_name}.csv', index=False)

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')

In [ ]:
submit.iloc[:,1:] = preds_tta
submit.head()

In [ ]:
submit.to_csv(f'./submits/tta_{file_name}.csv', index=False)